In [1]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
from IPython.display import display, Markdown, Latex
from datetime import datetime

In [2]:
API_KEY = os.environ['MISTRAL_API_KEY']

MODEL_ID = 'open-mixtral-8x22b'
MODEL_SEED = 3
MODEL_TEMPERATURE = 0.7

file_system_prompt_1 = open("../../system_prompt_1.txt", "r")
file_system_prompt_2 = open("../../system_prompt_2.txt", "r")
SYSTEM_PROMPT_1 = file_system_prompt_1.read()
SYSTEM_PROMPT_2 = file_system_prompt_2.read()
file_system_prompt_1.close()
file_system_prompt_2.close()

FILE_PATH_PROBLEM = '../../../../Datasets/IP_2_Choosing_Investment_Strategies/'
assert(FILE_PATH_PROBLEM != '../../../../Datasets/')
file_problem_description = open(FILE_PATH_PROBLEM + 'ProblemDescription.txt', 'r')
PROBLEM_DESCRIPTION = file_problem_description.read()
file_problem_description.close()


client = MistralClient(api_key=API_KEY)

print(f'Time of execution: {datetime.now()}')

Time of execution: 2024-05-22 12:38:17.459404


## Step 1 - Generate Mathematical Formulation 

In [3]:
messages_1 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_1),
    ChatMessage(role="user", content=PROBLEM_DESCRIPTION)
]

In [4]:
response_1 = client.chat(
    model=MODEL_ID,
    messages=messages_1,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE 
)

response_1_text = response_1.choices[0].message.content

In [5]:
Markdown(response_1_text)

This problem can be formulated as a mixed-integer linear programming (MILP) problem with chance constraints. Here are the parameters, decision variables, objective function, and constraints for this problem:

Parameters:

- \(A\): Vector of investment costs for each strategy, \(A = [80, 340, 410, 50, 180, 221, 15, 348, 191, 225]\)
- \(B\): Vector of possible budget scenarios, \(B = [1000, 1100, 900, 1200]\)
- \(P\): Vector of probabilities for each budget scenario, \(P = [0.55, 0.4, 0.04, 0.01]\)
- \(R\): Matrix of historic returns for each strategy, stored in the "investments_data.csv" file
- \(r\): Risk parameter, \(r = 2\)

Decision Variables:

- \(x_i\): Binary decision variable indicating whether to invest in strategy \(i\) (1) or not (0)
- \(y_j\): Auxiliary continuous variable representing the exceeded budget in scenario \(j\)
- \(z\): Auxiliary continuous variable representing the expected return of the investment portfolio

Objective Function:

The objective is to maximize the expected return of the investment portfolio while minimizing the risk, which can be represented by the mean-variance objective function:
\[ \text{maximize} \quad z - r \cdot \text{Var}(R \cdot x) \]

Constraints:

1. Budget constraints for each scenario:
\[ \sum_{i=1}^{10} A_i \cdot x_i + y_j \leq B_j \quad \forall j \in \{1,2,3,4\} \]
2. Probability of exceeding budget constraint:
\[ \sum_{j=1}^{4} P_j \cdot y_j \leq 0.4 \cdot \sum_{j=1}^{4} P_j \cdot B_j \]
3. Expected return calculation:
\[ z = \text{E}(R \cdot x) \]
4. Non-negativity constraints:
\[ y_j \geq 0 \quad \forall j \in \{1,2,3,4\} \]
5. Binary constraints for \(x_i\):
\[ x_i \in \{0, 1\} \quad \forall i \in \{1,2,\ldots,10\} \]

This MILP model can be solved using standard optimization solvers, such as CPLEX, Gurobi, or GLPK, to find the optimal investment strategy under the given constraints and uncertainties.

In [6]:
print(response_1_text)

This problem can be formulated as a mixed-integer linear programming (MILP) problem with chance constraints. Here are the parameters, decision variables, objective function, and constraints for this problem:

Parameters:

- \(A\): Vector of investment costs for each strategy, \(A = [80, 340, 410, 50, 180, 221, 15, 348, 191, 225]\)
- \(B\): Vector of possible budget scenarios, \(B = [1000, 1100, 900, 1200]\)
- \(P\): Vector of probabilities for each budget scenario, \(P = [0.55, 0.4, 0.04, 0.01]\)
- \(R\): Matrix of historic returns for each strategy, stored in the "investments_data.csv" file
- \(r\): Risk parameter, \(r = 2\)

Decision Variables:

- \(x_i\): Binary decision variable indicating whether to invest in strategy \(i\) (1) or not (0)
- \(y_j\): Auxiliary continuous variable representing the exceeded budget in scenario \(j\)
- \(z\): Auxiliary continuous variable representing the expected return of the investment portfolio

Objective Function:

The objective is to maximize the

## Step 2 - Generate the Pyomo Code

In [7]:
messages_2 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_2),
    ChatMessage(role="user", content=response_1_text)
]

In [8]:
response_2 = client.chat(
    model=MODEL_ID,
    messages=messages_2,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_2_text = response_2.choices[0].message.content

In [9]:
Markdown(response_2_text)

Here is the Python code using Pyomo to solve the optimization problem:

```python
import pyomo.environ as pyo
import pandas as pd
import numpy as np

# Sample data
A = np.array([80, 340, 410, 50, 180, 221, 15, 348, 191, 225])
B = np.array([1000, 1100, 900, 1200])
P = np.array([0.55, 0.4, 0.04, 0.01])
r = 2

# Load return data from csv file (assuming it has 10 rows and 4 columns)
R = pd.read_csv("investments_data.csv").to_numpy()

model = pyo.ConcreteModel()

# Define decision variables
model.x = pyo.Var(range(1, 11), domain=pyo.Binary)
model.y = pyo.Var(range(1, 5), domain=pyo.NonNegativeReals)
model.z = pyo.Var(domain=pyo.NonNegativeReals)

# Define objective function
def obj_rule(model):
    return model.z - r * sum(
        (R[i, j] * model.x[i + 1] - model.z) ** 2 * P[j] for i in range(10) for j in range(4)
    ) / 4

model.obj = pyo.Objective(rule=obj_rule, sense=pyo.maximize)

# Define constraints
def budget_constraints(model, j):
    return sum(A[i] * model.x[i + 1] + model.y[j + 1] for i in range(10)) <= B[j]

model.budget_constraints = pyo.Constraint(range(4), rule=budget_constraints)

def exceeded_budget_constraint(model):
    return sum(P[j] * model.y[j + 1] for j in range(4)) <= 0.4 * sum(P[j] * B[j] for j in range(4))

model.exceeded_budget_constraint = pyo.Constraint(rule=exceeded_budget_constraint)

def expected_return(model):
    return model.z == sum(R[i, j] * model.x[i + 1] * P[j] for i in range(10) for j in range(4)) / 4

model.expected_return = pyo.Constraint(rule=expected_return)

# Solve the model
solver = pyo.SolverFactory('glpk')
solver.solve(model)

# Print the optimal solution
print("Optimal solution:")
for i in range(1, 11):
    print(f"x{i} = {model.x[i].value}")
for j in range(1, 5):
    print(f"y{j} = {model.y[j].value}")
print(f"z = {model.z.value}")
```

This code defines the MILP problem using Pyomo, a Python-based open-source software package for modeling and solving mathematical optimization problems. It uses the GLPK solver to solve the problem, but you can change the solver to CPLEX or Gurobi by modifying the solver line. The sample data is used to define the parameters A, B, P, and R, and you can replace it with your actual data.

In [10]:
print(response_2_text)

Here is the Python code using Pyomo to solve the optimization problem:

```python
import pyomo.environ as pyo
import pandas as pd
import numpy as np

# Sample data
A = np.array([80, 340, 410, 50, 180, 221, 15, 348, 191, 225])
B = np.array([1000, 1100, 900, 1200])
P = np.array([0.55, 0.4, 0.04, 0.01])
r = 2

# Load return data from csv file (assuming it has 10 rows and 4 columns)
R = pd.read_csv("investments_data.csv").to_numpy()

model = pyo.ConcreteModel()

# Define decision variables
model.x = pyo.Var(range(1, 11), domain=pyo.Binary)
model.y = pyo.Var(range(1, 5), domain=pyo.NonNegativeReals)
model.z = pyo.Var(domain=pyo.NonNegativeReals)

# Define objective function
def obj_rule(model):
    return model.z - r * sum(
        (R[i, j] * model.x[i + 1] - model.z) ** 2 * P[j] for i in range(10) for j in range(4)
    ) / 4

model.obj = pyo.Objective(rule=obj_rule, sense=pyo.maximize)

# Define constraints
def budget_constraints(model, j):
    return sum(A[i] * model.x[i + 1] + model.y[j 

### Code Executability

In [13]:
import pyomo.environ as pyo
import pandas as pd
import numpy as np

# Sample data
A = np.array([80, 340, 410, 50, 180, 221, 15, 348, 191, 225])
B = np.array([1000, 1100, 900, 1200])
P = np.array([0.55, 0.4, 0.04, 0.01])
r = 2

# Load return data from csv file (assuming it has 10 rows and 4 columns)
R = pd.read_csv(FILE_PATH_PROBLEM + "investments_data.csv").to_numpy()

model = pyo.ConcreteModel()

# Define decision variables
model.x = pyo.Var(range(1, 11), domain=pyo.Binary)
model.y = pyo.Var(range(1, 5), domain=pyo.NonNegativeReals)
model.z = pyo.Var(domain=pyo.NonNegativeReals)

# Define objective function
def obj_rule(model):
    return model.z - r * sum(
        (R[i, j] * model.x[i + 1] - model.z) ** 2 * P[j] for i in range(10) for j in range(4)
    ) / 4

model.obj = pyo.Objective(rule=obj_rule, sense=pyo.maximize)

# Define constraints
def budget_constraints(model, j):
    return sum(A[i] * model.x[i + 1] + model.y[j + 1] for i in range(10)) <= B[j]

model.budget_constraints = pyo.Constraint(range(4), rule=budget_constraints)

def exceeded_budget_constraint(model):
    return sum(P[j] * model.y[j + 1] for j in range(4)) <= 0.4 * sum(P[j] * B[j] for j in range(4))

model.exceeded_budget_constraint = pyo.Constraint(rule=exceeded_budget_constraint)

def expected_return(model):
    return model.z == sum(R[i, j] * model.x[i + 1] * P[j] for i in range(10) for j in range(4)) / 4

model.expected_return = pyo.Constraint(rule=expected_return)

# Solve the model
solver = pyo.SolverFactory('glpk')
solver.solve(model)

# Print the optimal solution
print("Optimal solution:")
for i in range(1, 11):
    print(f"x{i} = {model.x[i].value}")
for j in range(1, 5):
    print(f"y{j} = {model.y[j].value}")
print(f"z = {model.z.value}")

2024-05-22 12:47:54,560 ERROR pyomo.core: Rule failed when generating expression for Objective obj with index None:
AttributeError: 'str' object has no attribute 'is_potentially_variable'
2024-05-22 12:47:54,561 ERROR pyomo.core: Constructing component 'obj' from data=None failed:
AttributeError: 'str' object has no attribute 'is_potentially_variable'


AttributeError: 'str' object has no attribute 'is_potentially_variable'

### Solution Correctness

In [14]:
import pyomo.environ as pyo
import pandas as pd
import numpy as np

# Sample data
A = np.array([80, 340, 410, 50, 180, 221, 15, 348, 191, 225])
B = np.array([1000, 1100, 900, 1200])
P = np.array([0.55, 0.4, 0.04, 0.01])
r = 2

# Load return data from csv file (assuming it has 10 rows and 4 columns)
R = pd.read_csv(FILE_PATH_PROBLEM + "investments_data.csv", sep=';').to_numpy()

model = pyo.ConcreteModel()

# Define decision variables
model.x = pyo.Var(range(1, 11), domain=pyo.Binary)
model.y = pyo.Var(range(1, 5), domain=pyo.NonNegativeReals)
model.z = pyo.Var(domain=pyo.NonNegativeReals)

# Define objective function
def obj_rule(model):
    return model.z - r * sum(
        (R[i, j] * model.x[i + 1] - model.z) ** 2 * P[j] for i in range(10) for j in range(4)
    ) / 4

model.obj = pyo.Objective(rule=obj_rule, sense=pyo.maximize)

# Define constraints
def budget_constraints(model, j):
    return sum(A[i] * model.x[i + 1] + model.y[j + 1] for i in range(10)) <= B[j]

model.budget_constraints = pyo.Constraint(range(4), rule=budget_constraints)

def exceeded_budget_constraint(model):
    return sum(P[j] * model.y[j + 1] for j in range(4)) <= 0.4 * sum(P[j] * B[j] for j in range(4))

model.exceeded_budget_constraint = pyo.Constraint(rule=exceeded_budget_constraint)

def expected_return(model):
    return model.z == sum(R[i, j] * model.x[i + 1] * P[j] for i in range(10) for j in range(4)) / 4

model.expected_return = pyo.Constraint(rule=expected_return)

# Solve the model
solver = pyo.SolverFactory('glpk')
solver.solve(model)

# Print the optimal solution
print("Optimal solution:")
for i in range(1, 11):
    print(f"x{i} = {model.x[i].value}")
for j in range(1, 5):
    print(f"y{j} = {model.y[j].value}")
print(f"z = {model.z.value}")

RuntimeError: Selected solver is unable to handle objective functions with quadratic terms. Objective at issue: obj.